In [ ]:
# =====================================================
# BLOCK 3: CROSSTAB – RISK vs FAILURE STAGE
# =====================================================

risk_vs_failure = pd.crosstab(
    df_scenario["risk_category"],
    df_scenario["tipping_point"]
)

risk_vs_failure


In [ ]:
# =====================================================
# BLOCK 4: BAR PLOT – RISK CATEGORY vs FAILURE
# =====================================================

risk_vs_failure.plot(
    kind="bar",
    figsize=(10,6)
)

plt.xlabel("Risk Category")
plt.ylabel("Number of Districts")
plt.title("Failure Stage Distribution by Risk Category")
plt.xticks(rotation=0)
plt.legend(title="Failure Stage")
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# BLOCK 1: FILTER HIGH-RISK DISTRICTS (SURGE SCENARIO)
# =====================================================

high_risk_df = df_scenario[
    df_scenario["risk_category"] == "High Risk"
].copy()

high_risk_df[
    ["district", "risk_category", "stress_surge"]
]


In [ ]:
# =====================================================
# ADD STATE-LEVEL EMERGENCY BED POOL
# =====================================================

emergency_pool = 0.10 * high_risk_df["total_beds"].sum()  # 10% surge beds


In [ ]:
# =====================================================
# BLOCK 1: RISK-AWARE SURGE OPTIMIZATION (LP)
# =====================================================

import pulp
import numpy as np

# Scenario dataframe (safe copy)
df_scenario = df_features.copy()

# Surge demand
df_scenario["ip_surge"] = df_scenario["ip_demand"] * 1.60

# Inputs
districts = df_scenario["district"].tolist()

ip_surge = dict(zip(df_scenario["district"], df_scenario["ip_surge"]))
risk = dict(zip(df_scenario["district"], df_scenario["demand_risk_index"]))
current_beds = dict(zip(df_scenario["district"], df_scenario["total_beds"]))

TOTAL_STATE_BEDS = sum(current_beds.values())

# Baseline unmet demand (before optimization)
unmet_before = {
    d: max(ip_surge[d] - current_beds[d], 0)
    for d in districts
}

# LP model
model = pulp.LpProblem(
    "Risk_Aware_Surge_Bed_Optimization",
    pulp.LpMaximize
)

# Decision variables
beds_alloc = {
    d: pulp.LpVariable(f"beds_alloc_{d}", lowBound=0)
    for d in districts
}

unmet_after = {
    d: pulp.LpVariable(
        f"unmet_after_{d}",
        lowBound=0,
        upBound=unmet_before[d]  # cannot worsen
    )
    for d in districts
}

# Objective: risk-weighted reduction in unmet demand
model += pulp.lpSum(
    risk[d] * (unmet_before[d] - unmet_after[d])
    for d in districts
)

# Constraints
for d in districts:
    model += unmet_after[d] >= ip_surge[d] - beds_alloc[d]

# State-level bed pool constraint
model += pulp.lpSum(beds_alloc[d] for d in districts) <= TOTAL_STATE_BEDS

# Solve
model.solve(pulp.PULP_CBC_CMD(msg=False))

# Store results
df_scenario["beds_allocated_surge_optimal"] = df_scenario["district"].apply(
    lambda d: beds_alloc[d].varValue
)

df_scenario["unmet_surge_before"] = df_scenario["district"].apply(
    lambda d: unmet_before[d]
)

df_scenario["unmet_surge_after"] = df_scenario["district"].apply(
    lambda d: unmet_after[d].varValue
)

df_scenario[
    [
        "district",
        "ip_surge",
        "total_beds",
        "beds_allocated_surge_optimal",
        "unmet_surge_before",
        "unmet_surge_after"
    ]
]


In [ ]:
# =====================================================
# BLOCK 2: VISUALIZATION (BEFORE vs AFTER)
# =====================================================

import matplotlib.pyplot as plt

# Sort for clarity
plot_df = df_scenario.sort_values(
    "unmet_surge_before", ascending=False
)

districts = plot_df["district"]
before = plot_df["unmet_surge_before"]
after = plot_df["unmet_surge_after"]

x = np.arange(len(districts))
width = 0.35

plt.figure(figsize=(15,6))

plt.bar(
    x - width/2,
    before,
    width,
    label="Before Optimization (Surge)",
    color="steelblue"
)

plt.bar(
    x + width/2,
    after,
    width,
    label="After Optimization (Surge)",
    color="darkorange"
)

plt.xticks(x, districts, rotation=45)
plt.ylabel("Unmet Inpatient Demand")
plt.xlabel("District")
plt.title("Surge Scenario: Unmet Inpatient Demand Before vs After Optimization")
plt.legend()
plt.grid(axis="y", linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# =====================================================
# KERALA HEALTHCARE STRESS TESTING – EXTREME SURGES
# =====================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------------
# WORKING DATAFRAME
# -------------------------------
df_stress = df_features.copy()

# -------------------------------
# CONFIGURATION
# -------------------------------
SURGE_SCENARIOS = {
    "Baseline (1.0x)": 1.0,
    "Mild Surge (1.2x)": 1.2,
    "Moderate Surge (1.4x)": 1.4,
    "Severe Surge (1.6x)": 1.6,
    "Extreme Surge (1.8x)": 1.8,
    "Collapse Zone (2.0x)": 2.0
}

RISK_COL = "demand_risk_index"

# -------------------------------
# BASE DEMAND & CAPACITY
# -------------------------------
df_stress["bed_capacity"] = df_stress["total_beds"]
df_stress["icu_capacity"] = df_stress["icu_beds"]
df_stress["vent_capacity"] = (
    df_stress["ventilators_invasive"] +
    df_stress["ventilators_non_invasive"]
)

# Demand proxies (standard in healthcare planning)
df_stress["bed_demand_base"] = df_stress["ip_demand"]
df_stress["icu_demand_base"] = df_stress["ip_demand"] * 0.15
df_stress["vent_demand_base"] = df_stress["ip_demand"] * 0.05

# -------------------------------
# STRESS TEST LOOP
# -------------------------------
results = []

for scenario, factor in SURGE_SCENARIOS.items():

    bed_demand = df_stress["bed_demand_base"] * factor
    icu_demand = df_stress["icu_demand_base"] * factor
    vent_demand = df_stress["vent_demand_base"] * factor

    bed_unmet = np.maximum(bed_demand - df_stress["bed_capacity"], 0)
    icu_unmet = np.maximum(icu_demand - df_stress["icu_capacity"], 0)
    vent_unmet = np.maximum(vent_demand - df_stress["vent_capacity"], 0)

    results.append({
        "Scenario": scenario,
        "Surge_Factor": factor,
        "Total_Bed_Unmet": bed_unmet.sum(),
        "Total_ICU_Unmet": icu_unmet.sum(),
        "Total_Vent_Unmet": vent_unmet.sum(),
        "High_Risk_Districts": (bed_unmet > 0).sum()
    })

stress_df = pd.DataFrame(results)

# -------------------------------
# DISPLAY SUMMARY TABLE
# -------------------------------
stress_df


In [ ]:
# =====================================================
# PLOT: UNMET DEMAND vs SURGE LEVEL
# =====================================================

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))

plt.plot(
    stress_df["Surge_Factor"],
    stress_df["Total_Bed_Unmet"],
    marker="o",
    linewidth=2,
    label="Beds"
)

plt.plot(
    stress_df["Surge_Factor"],
    stress_df["Total_ICU_Unmet"],
    marker="s",
    linewidth=2,
    label="ICU"
)

plt.plot(
    stress_df["Surge_Factor"],
    stress_df["Total_Vent_Unmet"],
    marker="^",
    linewidth=2,
    label="Ventilators"
)

# Labels & title
plt.xlabel("Surge Multiplier (Demand Increase)")
plt.ylabel("Total Unmet Demand")
plt.title("Kerala Healthcare Stress Test: Capacity Failure Curve")

# Grid & legend
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()

plt.tight_layout()
plt.show()




SURGE LP Formulation(surge scenarios)